In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r'data/ds_salaries.csv')

# PREPROCESS

In [7]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


How to check the columns we are working with.

In [8]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

How to check how many rows and columns we have.

In [9]:
df.shape #the first number will be number of rows and the other one columns.

(3755, 11)

How to check the type of data we have in each column.

In [10]:
df.dtypes #whole numbers are represented with int64, and strings as objects.

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

#### The first thing we need to do is check if we have null values.

In [11]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

#### Then we check for duplicates

In [12]:
print(df.duplicated().sum())


1171


Once we find there are duplicates, we can check what are they so we can decide what to do with them.

In [13]:
duplicates = pd.DataFrame(df.value_counts())
duplicates[duplicates['count']>1]

count
work_year experience_level employment_type job_title            salary salary_currency salary_in_usd employee_residence remote_ratio company_location company_size       
2022      SE               FT              Data Scientist       191475 USD             191475        US                 100          US               M                21
                                                                141525 USD             141525        US                 100          US               M                21
2023      SE               FT              Data Engineer        252000 USD             252000        US                 0            US               M                13
                                                                129000 USD             129000        US                 0            US               M                13
2022      SE               FT              Data Engineer        130000 USD             130000        US                 0            US               M                12
...                                                                                                                                                                   ...
                                           Data Scientist       119300 USD             119300        US                 0            US               L                 2
2023      SE               FT              Data Quality Analyst 100000 USD             100000        US                 0            US               M                 2
                                                                80000  USD             80000         US                 0            US               M                 2
2022      EN               FT              Data Analyst         50000  USD             50000         US                 50           US               L                 2
2023      MI               FT              Data Analyst         85000  USD             85000         US                 0            US               M                 2

[544 rows x 1 columns]

We have decided to delete duplicates, as in this particular case it might alter results.

In [14]:
df.drop_duplicates(inplace=True)

We then check our new dataset without the duplicates.

In [15]:
df.shape

(2584, 11)

We see that before we had 3755 rows and now we have 2584.

# EXPLORATORY DATA ANALYSIS (EDA)

We print one row to see how the data is presented.

In [16]:
df.head(1)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L


#### If we were not given instructions, it's very important to ask some questions ourselves.

What job position pays more is USD?

In [25]:
# Group the data by job title to find the maximum salary in each group.
max_salaries_by_job_title = df.groupby('job_title')['salary_in_usd'].max()

# To find the job with the highest salary
highest_paying_job = max_salaries_by_job_title.idxmax()  # Returns the job with the highest salary
highest_salary = max_salaries_by_job_title.max()        # Returns the highest salary

print(f"The highest paying job is: {highest_paying_job} with a salary of {highest_salary}")

The highest paying job is: Research Scientist with a salary of 450000


What country has the most amount of large companies?

In [26]:
# Filter the data for large companies
large_companies_df = df[df['company_size'] == 'L']

# Group by company location and count the number of large companies in each country
large_companies_by_country = large_companies_df.groupby('company_location').size()

# Find the country with the most large companies
country_with_most_large_companies = large_companies_by_country.idxmax()  # Country with most large companies
most_large_company_count = large_companies_by_country.max()              # Number of large companies in that country

print(f"The country with the most large companies is: {country_with_most_large_companies}, with {most_large_company_count} large companies.")


The country with the most large companies is: US, with 220 large companies.


Is the level of experience proportional to the salary?

What is the average salary for each level of experience?

In [32]:
# Calculate the average salary for each experience level
df.groupby('experience_level')['salary_in_usd'].mean().round(2)

experience_level
EN     72648.69
EX    191078.21
MI    101828.78
SE    153897.44
Name: salary_in_usd, dtype: float64

What is the average salary percentage increase between each level of experience?

In [34]:
# Calculate the average salary for each experience level
average_salary_by_experience = df.groupby('experience_level')['salary_in_usd'].mean().round(2)

# Group the results
order_experience_level = ['EN', 'MI', 'SE', 'EX']

#Use reindex command 
average_salary_by_experience = average_salary_by_experience.reindex(order_experience_level)

#Use pct.change to calculate percentage variation per row
percentage_increase = average_salary_by_experience.pct_change() * 100

#show the dataframe
percentage_increase.round(2)



experience_level
EN      NaN
MI    40.17
SE    51.13
EX    24.16
Name: salary_in_usd, dtype: float64

What company size pays more?